In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
path = '/content/drive/MyDrive/KoGPT-2 프로젝트'

In [1]:
!pip install transformers
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.2 MB 4.1 MB/s 
     |████████████████████████████████| 86 kB 5.5 MB/s 
     |████████████████████████████████| 596 kB 62.3 MB/s 
     |████████████████████████████████| 6.6 MB 51.1 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 346 kB 4.2 MB/s 
     |████████████████████████████████| 212 kB 46.2 MB/s 
     |████████████████████████████████| 140 kB 69.0 MB/s 
     |████████████████████████████████| 1.1 MB 34.9 MB/s 
     |████████████████████████████████| 86 kB 4.0 MB/s 
     |████████████████████████████████| 127 kB 75.1 MB/s 
     |████████████████████████████████| 271 kB 61.9 MB/s 
     |███████████████

In [2]:
!gdown 1TBrPvWuvAv71nkRtvvZlrWEcj62HZrtF

Downloading...
From: https://drive.google.com/uc?id=1TBrPvWuvAv71nkRtvvZlrWEcj62HZrtF
To: /content/시사랑 10만 최종 데이터.csv
100% 80.4M/80.4M [00:00<00:00, 269MB/s]


In [5]:
from transformers import AutoModelForCausalLM, TrainingArguments, Trainer, DataCollatorForLanguageModeling, DataCollatorWithPadding
from transformers import AutoConfig, AutoModel, AutoTokenizer, GPT2LMHeadModel, PreTrainedTokenizerFast
from datasets import load_metric, Dataset
import torch
import numpy as np
import pandas as pd

In [6]:
csv_name = '/content/시사랑 10만 최종 데이터.csv'
df = pd.read_csv(csv_name)

In [7]:
df.dropna(inplace=True)
df.reset_index(inplace=True)

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 106371 entries, 0 to 106370
Data columns (total 5 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   index       106371 non-null  int64 
 1   Unnamed: 0  106371 non-null  int64 
 2   title       106371 non-null  object
 3   content     106371 non-null  object
 4   author      106371 non-null  object
dtypes: int64(2), object(3)
memory usage: 4.1+ MB


In [9]:
# sample_num = 20000
# df = df[:sample_num]

In [10]:
dataset = Dataset.from_pandas(df)

In [11]:
dataset = dataset.remove_columns(['index', 'Unnamed: 0', 'title', 'author'])

In [12]:
dataset = dataset.train_test_split(test_size=0.2)

In [13]:
model_name = "skt/kogpt2-base-v2"

In [14]:
tokenizer = AutoTokenizer.from_pretrained(model_name,
    bos_token='</s>', eos_token='</s>', unk_token='<unk>',
    pad_token='<pad>', mask_token='<mask>')

# tokenizer = PreTrainedTokenizerFast.from_pretrained(model_name,
#     bos_token='</s>', eos_token='</s>', unk_token='<unk>',
#     pad_token='<pad>', mask_token='<mask>')

# tokenizer = AutoTokenizer.from_pretrained(model_name)

Downloading:   0%|          | 0.00/0.98k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.69M [00:00<?, ?B/s]

In [ ]:
model = AutoModelForCausalLM.from_pretrained(model_name)

# model = GPT2LMHeadModel.from_pretrained(model_name)
# model = AutoModelForCausalLM.from_pretrained(path + '/results/checkpoint-3991/')

Downloading:   0%|          | 0.00/490M [00:00<?, ?B/s]

In [ ]:
# tokenizer.pad_token = tokenizer.eos_token

In [16]:
dataset

DatasetDict({
    train: Dataset({
        features: ['content'],
        num_rows: 85096
    })
    test: Dataset({
        features: ['content'],
        num_rows: 21275
    })
})

In [17]:
def preprocess_function(examples):
    return tokenizer(examples["content"], truncation=True, padding='max_length', max_length=256)
    # return tokenizer([" ".join(x) for x in examples["content"]], truncation=True, padding='max_length', max_length=256)

In [18]:
tokenized_dataset = dataset.map(
    preprocess_function,
    batched=True,
    num_proc=4,
    remove_columns=dataset["train"].column_names,
)

#0:   0%|          | 0/22 [00:00<?, ?ba/s]

#1:   0%|          | 0/22 [00:00<?, ?ba/s]

#2:   0%|          | 0/22 [00:00<?, ?ba/s]

#3:   0%|          | 0/22 [00:00<?, ?ba/s]

#0:   0%|          | 0/6 [00:00<?, ?ba/s]

#1:   0%|          | 0/6 [00:00<?, ?ba/s]

#2:   0%|          | 0/6 [00:00<?, ?ba/s]

#3:   0%|          | 0/6 [00:00<?, ?ba/s]

In [19]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 85096
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 21275
    })
})

In [20]:
enc = tokenized_dataset['train']['input_ids'][0]
tokenizer.decode(enc)

'여가수에게 한 곡조 잘 뽑기 위하여수십 번 아니 수천 수만 번피를 토해야 하리그렇게 거르고 거른마지막 가락 그 가락이환상의 나라착각의 시민을 잡는다안개 짙은 거리에서우리는 마약중독자가 되고몽유병자가 되고 그대는 세기의 최면술사원격조종에도 문을 열고내장을 드러내는우리는 어쩔 수 없는 자동판매기안개는 다시 는개로 내리고사내들은 사타구니에 문신을 뜬다검은 장미 봉우리를 하나하나깊이깊이 박아넣는다</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>'

In [21]:
block_size = 256

def group_texts(examples):
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

In [22]:
lm_dataset = tokenized_dataset.map(group_texts, batched=True, num_proc=4)

#0:   0%|          | 0/22 [00:00<?, ?ba/s]

#1:   0%|          | 0/22 [00:00<?, ?ba/s]

#2:   0%|          | 0/22 [00:00<?, ?ba/s]

#3:   0%|          | 0/22 [00:00<?, ?ba/s]

#0:   0%|          | 0/6 [00:00<?, ?ba/s]

#1:   0%|          | 0/6 [00:00<?, ?ba/s]

#2:   0%|          | 0/6 [00:00<?, ?ba/s]

#3:   0%|          | 0/6 [00:00<?, ?ba/s]

In [23]:
lm_dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 85096
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 21275
    })
})

In [ ]:
enc = lm_dataset['train']['input_ids'][0]
tokenizer.decode(enc)

'밤하늘 위로 별똥별 떨어지고 멍석 깔고 마당에 앉아 수박 먹던 그 시절 귀뚜라미 개구리 정신없이 울어대고 이웃들 정다운 마음이 모여 하하 호호 친구들 투명한 마음이 전달되며 속닥속닥 다시 찾은 이곳 내 밟은 흙 간데없고 차가운 회색의 거리 우뚝우뚝 솟은 빌딩들 거리마다 간판들 춤추고 내 친구들 어디 가서 만나랴 내 이웃은 아직도 있는가</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><p

In [ ]:
# tokenizer.pad_token = tokenizer.eos_token

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)
# data_collator = DataCollatorWithPadding(tokenizer=tokenizer, padding='max_length', max_length=16)

In [ ]:
training_args = TrainingArguments(
    output_dir=path+"/results",
    save_strategy="epoch",
    save_total_limit=3,
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    per_device_eval_batch_size=8,
    per_device_train_batch_size=8,
    num_train_epochs=6.0,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_dataset["train"],
    eval_dataset=lm_dataset["test"],
    data_collator=data_collator,
)

In [ ]:
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 85096
  Num Epochs = 6
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 63822


Epoch,Training Loss,Validation Loss
1,4.671700,4.577861
2,4.509700,4.497214
3,4.385400,4.460610
4,4.296700,4.435899
5,4.198000,4.419502
6,4.131800,4.418070


***** Running Evaluation *****
  Num examples = 21275
  Batch size = 8
Saving model checkpoint to /content/drive/MyDrive/KoGPT-2 프로젝트/results/checkpoint-10637
Configuration saved in /content/drive/MyDrive/KoGPT-2 프로젝트/results/checkpoint-10637/config.json
Model weights saved in /content/drive/MyDrive/KoGPT-2 프로젝트/results/checkpoint-10637/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 21275
  Batch size = 8
Saving model checkpoint to /content/drive/MyDrive/KoGPT-2 프로젝트/results/checkpoint-21274
Configuration saved in /content/drive/MyDrive/KoGPT-2 프로젝트/results/checkpoint-21274/config.json
Model weights saved in /content/drive/MyDrive/KoGPT-2 프로젝트/results/checkpoint-21274/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 21275
  Batch size = 8
Saving model checkpoint to /content/drive/MyDrive/KoGPT-2 프로젝트/results/checkpoint-31911
Configuration saved in /content/drive/MyDrive/KoGPT-2 프로젝트/results/checkpoint-31911/con

TrainOutput(global_step=63822, training_loss=4.387000026991574, metrics={'train_runtime': 21579.6115, 'train_samples_per_second': 23.66, 'train_steps_per_second': 2.958, 'total_flos': 6.6704720265216e+16, 'train_loss': 4.387000026991574, 'epoch': 6.0})

In [ ]:
tokenizer.save_pretrained(path + '/saved')
model.save_pretrained(path + '/saved')

tokenizer config file saved in /content/drive/MyDrive/KoGPT-2 프로젝트/saved/tokenizer_config.json
Special tokens file saved in /content/drive/MyDrive/KoGPT-2 프로젝트/saved/special_tokens_map.json
Configuration saved in /content/drive/MyDrive/KoGPT-2 프로젝트/saved/config.json
Model weights saved in /content/drive/MyDrive/KoGPT-2 프로젝트/saved/pytorch_model.bin


In [ ]:
tokenizer = AutoTokenizer.from_pretrained(path + '/saved')

In [ ]:
model = AutoModelForCausalLM.from_pretrained(path + '/saved')

In [ ]:
text = '그대여'
input_ids = tokenizer.encode(text, return_tensors='pt')
gen_ids = model.generate(input_ids,
                           max_length=128,
                           repetition_penalty=2.0,
                           pad_token_id=tokenizer.pad_token_id,
                           eos_token_id=tokenizer.eos_token_id,
                           bos_token_id=tokenizer.bos_token_id,
                           use_cache=True)
generated = tokenizer.decode(gen_ids[0])
print(generated)

그대여 당신의 그리움은 다시 안녕하신가요 별빛처럼 빛나는 그대의 마음에서 당신을 향한 애태움으로 채워지는 것은 오직 순결함뿐입니다 언제나 변하지 않는 순수함으로 그리고 진실하게 마음을 다스리도록 사랑하는 이에게 고백하는 것 그것은 처음 사랑의 모습인 것을 그래서 아름다운 날에요한 밤하늘에 별이 되어버린 날들의 기억 속에서 봅시다 이렇게 아름다움이면 천년의 시작과 끝까지도 그렇게 빛나고 있습니다가 이제야 알게 된 아름다움의 순간들은 어쩌면 마음의 중심에서부터 이미 어두움을 걷어내고 있는지도 모릅니까 나의 마음에 빛이 비치는 곳마다 별의 중심에 서있는 것처럼 이제는


In [ ]:
txt = '서글픈 장미 겨울 바람에 보송보송한 햇살 그리워지던 꿈조각 연초록 눈빛 벙긋벙긋'

In [ ]:
tokenized = tokenizer.tokenize(txt)
tokenized

['▁서',
 '글',
 '픈',
 '▁장미',
 '▁겨울',
 '▁바람에',
 '▁보',
 '송',
 '보',
 '송',
 '한',
 '▁햇',
 '살',
 '▁그리워',
 '지던',
 '▁꿈',
 '조각',
 '▁연',
 '초',
 '록',
 '▁눈',
 '빛',
 '▁벙',
 '긋',
 '벙',
 '긋']

In [ ]:
enc = tokenizer(txt)
enc.input_ids

[9071,
 6951,
 8689,
 28984,
 11457,
 17072,
 9049,
 7832,
 7652,
 7832,
 8704,
 15749,
 7760,
 27094,
 32594,
 12582,
 17266,
 9081,
 8381,
 7427,
 9847,
 7702,
 43860,
 6955,
 7629,
 6955]

In [ ]:
tokenizer.decode(enc.input_ids)

'서글픈 장미 겨울 바람에 보송보송한 햇살 그리워지던 꿈조각 연초록 눈빛 벙긋벙긋'